In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_openai import ChatOpenAI


In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
KEY = os.getenv("OPENAI_API_KEY")

In [5]:
# llm = ChatOpenAI(
#     openai_api_key=KEY,
#     model_name="openai/gpt-3.5-turbo",
#     temperature=0.7
# )

In [6]:
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=KEY,
    temperature=0.7
)

In [7]:
KEY

'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'

In [8]:
# llm

In [9]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
# from langchain.callbacks import get_openai_callback
from langchain_community.callbacks.manager import get_openai_callback
import PyPDF2

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    
    "2": {
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    
    "3": {
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    
}

In [11]:
template="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [12]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=template
    )


In [13]:
# quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [14]:
from langchain.schema.runnable import RunnablePassthrough

# Quiz Chain
quiz_chain = quiz_generation_prompt | llm

In [15]:
template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of teh question and give a complete analysis of the quiz if the students
will be able to unserstand the questions and answer them. Only use at max 50 words for complexity analysis. 
if the quiz is not at par with the cognitive and analytical abilities of the students,\
update tech quiz questions which needs to be changed  and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}


Check from an expert English Writer of the above quiz:
"""

In [16]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=template2
)

In [17]:
# Quiz Review Chain
review_chain = quiz_evaluation_prompt | llm

In [18]:
# review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [19]:
from langchain.schema.runnable import RunnableLambda, RunnableMap


# Define Final RunnableMap
generate_evaluate_chain = RunnableMap({
    "quiz": quiz_chain,  # Runs quiz_chain and stores output as "quiz"
    "subject": lambda x: x["subject"]  # Pass "subject" directly
}) | review_chain


In [20]:
# ✅ Fix: Ensure both outputs are returned
def run_review_chain(input_data):
    """Runs review_chain and ensures both quiz and review are returned"""
    quiz_output = input_data["quiz"]
    review_output = review_chain.invoke({"quiz": quiz_output, "subject": input_data["subject"]})
    return {"quiz": quiz_output, "review": review_output}

In [21]:
# ✅ Fix: Ensure both quiz and review are captured
generate_evaluate_chain = RunnableMap({
    "quiz": quiz_chain,  # Generate Quiz
    "subject": lambda x: x["subject"],  # Pass subject
}) | RunnableLambda(run_review_chain)  # Run Review and Return Both


In [22]:
file_path = r"/home/harshit433/mcqgen/data.txt"

In [23]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [24]:
TEXT

'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]\n\nFrom a theoretical viewpoint, probably ap

In [25]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [26]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [27]:
# Execute with Callback
with get_openai_callback() as cb:
    response = generate_evaluate_chain.invoke({
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    })

In [28]:
print(f"Total Tokens: {cb.total_tokens}") 
print(f"Prompt Tokens: {cb.prompt_tokens}") 
print(f"Completion Tokens: {cb.completion_tokens}" )
print(f"Total Cost: {cb.total_cost}") 

Total Tokens: 2339
Prompt Tokens: 1868
Completion Tokens: 471
Total Cost: 0.0


In [29]:
response

{'quiz': AIMessage(content='\n{\n  "1": {\n    "mcq": "What is machine learning focused on?",\n    "options": {\n      "a": "Development of neural networks",\n      "b": "Development and study of statistical algorithms",\n      "c": "Development of computer vision",\n      "d": "Development of email filtering systems"\n    },\n    "correct": "b"\n  },\n  "2": {\n    "mcq": "Which field of study focuses on exploratory data analysis via unsupervised learning?",\n    "options": {\n      "a": "Machine learning",\n      "b": "Deep learning",\n      "c": "Natural language processing",\n      "d": "Data mining"\n    },\n    "correct": "d"\n  },\n  "3": {\n    "mcq": "Who coined the term \'machine learning\' in 1959?",\n    "options": {\n      "a": "Donald Hebb",\n      "b": "Tom M. Mitchell",\n      "c": "Arthur Samuel",\n      "d": "Alan Turing"\n    },\n    "correct": "c"\n  },\n  "4": {\n    "mcq": "What was the purpose of the experimental \'learning machine\' Cybertron developed by Raythe

In [30]:
quiz = response.get("quiz")

In [31]:
import json
from langchain_core.messages.ai import AIMessage

In [32]:
quiz

AIMessage(content='\n{\n  "1": {\n    "mcq": "What is machine learning focused on?",\n    "options": {\n      "a": "Development of neural networks",\n      "b": "Development and study of statistical algorithms",\n      "c": "Development of computer vision",\n      "d": "Development of email filtering systems"\n    },\n    "correct": "b"\n  },\n  "2": {\n    "mcq": "Which field of study focuses on exploratory data analysis via unsupervised learning?",\n    "options": {\n      "a": "Machine learning",\n      "b": "Deep learning",\n      "c": "Natural language processing",\n      "d": "Data mining"\n    },\n    "correct": "d"\n  },\n  "3": {\n    "mcq": "Who coined the term \'machine learning\' in 1959?",\n    "options": {\n      "a": "Donald Hebb",\n      "b": "Tom M. Mitchell",\n      "c": "Arthur Samuel",\n      "d": "Alan Turing"\n    },\n    "correct": "c"\n  },\n  "4": {\n    "mcq": "What was the purpose of the experimental \'learning machine\' Cybertron developed by Raytheon Compan

In [33]:
# Convert AIMessage content to JSON
quiz_dict = json.loads(quiz.content)  # Parse string into dictionary

# Print formatted JSON output
print(json.dumps(quiz_dict, indent=4))

{
    "1": {
        "mcq": "What is machine learning focused on?",
        "options": {
            "a": "Development of neural networks",
            "b": "Development and study of statistical algorithms",
            "c": "Development of computer vision",
            "d": "Development of email filtering systems"
        },
        "correct": "b"
    },
    "2": {
        "mcq": "Which field of study focuses on exploratory data analysis via unsupervised learning?",
        "options": {
            "a": "Machine learning",
            "b": "Deep learning",
            "c": "Natural language processing",
            "d": "Data mining"
        },
        "correct": "d"
    },
    "3": {
        "mcq": "Who coined the term 'machine learning' in 1959?",
        "options": {
            "a": "Donald Hebb",
            "b": "Tom M. Mitchell",
            "c": "Arthur Samuel",
            "d": "Alan Turing"
        },
        "correct": "c"
    },
    "4": {
        "mcq": "What was the purp

In [34]:
type(quiz)

langchain_core.messages.ai.AIMessage

In [35]:
# ✅ Print the complete response
print("Generated Quiz:\n", response["quiz"])
print("\nEvaluation Review:\n", response["review"])


Generated Quiz:
 content='\n{\n  "1": {\n    "mcq": "What is machine learning focused on?",\n    "options": {\n      "a": "Development of neural networks",\n      "b": "Development and study of statistical algorithms",\n      "c": "Development of computer vision",\n      "d": "Development of email filtering systems"\n    },\n    "correct": "b"\n  },\n  "2": {\n    "mcq": "Which field of study focuses on exploratory data analysis via unsupervised learning?",\n    "options": {\n      "a": "Machine learning",\n      "b": "Deep learning",\n      "c": "Natural language processing",\n      "d": "Data mining"\n    },\n    "correct": "d"\n  },\n  "3": {\n    "mcq": "Who coined the term \'machine learning\' in 1959?",\n    "options": {\n      "a": "Donald Hebb",\n      "b": "Tom M. Mitchell",\n      "c": "Arthur Samuel",\n      "d": "Alan Turing"\n    },\n    "correct": "c"\n  },\n  "4": {\n    "mcq": "What was the purpose of the experimental \'learning machine\' Cybertron developed by Raytheon

In [36]:
# This is an Overall Chain where we run the two chains in Sequence
# generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
#                                         output_variables=["quiz", "review"], verbose=True,)

In [37]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [38]:
quiz_table_data

[{'MCQ': 'What is machine learning focused on?',
  'Choices': 'a: Development of neural networks | b: Development and study of statistical algorithms | c: Development of computer vision | d: Development of email filtering systems',
  'Correct': 'b'},
 {'MCQ': 'Which field of study focuses on exploratory data analysis via unsupervised learning?',
  'Choices': 'a: Machine learning | b: Deep learning | c: Natural language processing | d: Data mining',
  'Correct': 'd'},
 {'MCQ': "Who coined the term 'machine learning' in 1959?",
  'Choices': 'a: Donald Hebb | b: Tom M. Mitchell | c: Arthur Samuel | d: Alan Turing',
  'Correct': 'c'},
 {'MCQ': "What was the purpose of the experimental 'learning machine' Cybertron developed by Raytheon Company in the early 1960s?",
  'Choices': 'a: To play chess | b: To analyze sonar signals, electrocardiograms, and speech patterns | c: To recognize patterns in nature | d: To study human cognitive processes',
  'Correct': 'b'},
 {'MCQ': 'What is one of the 

In [39]:
df=pd.DataFrame(quiz_table_data)

In [40]:
df

,MCQ,Choices,Correct
0,What is machine learning focused on?,a: Development of neural networks | b: Develop...,b
1,Which field of study focuses on exploratory da...,a: Machine learning | b: Deep learning | c: Na...,d
2,Who coined the term 'machine learning' in 1959?,a: Donald Hebb | b: Tom M. Mitchell | c: Arthu...,c
3,What was the purpose of the experimental 'lear...,a: To play chess | b: To analyze sonar signals...,b
4,What is one of the objectives of modern-day ma...,a: To develop new programming languages | b: T...,b


In [41]:
df.to_csv("machinelearning.csv",index=False)

In [47]:
from datetime import datetime

datetime.now()

datetime.datetime(2025, 3, 13, 12, 40, 41, 812762)